In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL 저장하기") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "512m")\
    .config("spark.jars", "/jars/redshift-jdbc42-2.1.0.19.jar")\
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.adaptive.enabled", False) \
    .enableHiveSupport() \
    .getOrCreate()

23/10/04 02:24:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Redshift와 연결해서 DataFrame으로 로딩하기
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

In [3]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
df_join = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")
df_join.show(10)


+------+--------------------+--------+--------------------+--------------------+
|userid|           sessionid| channel|           sessionid|                  ts|
+------+--------------------+--------+--------------------+--------------------+
|  1501|0135456d6a3c1051f...|  Google|0135456d6a3c1051f...|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...|Facebook|01a416a7e28d0d229...|2019-05-26 14:23:...|
|   243|0226aa5193c66d990...|  Google|0226aa5193c66d990...|2019-07-01 23:04:...|
|  2776|029bf49b584c641f0...|Facebook|029bf49b584c641f0...|2019-11-11 20:37:...|
|   939|02b8d6c2775b756de...|  Google|02b8d6c2775b756de...|2019-09-01 15:29:...|
|  2133|02ea66ee24d57e285...| Organic|02ea66ee24d57e285...| 2019-10-28 22:50:40|
|   771|02f6364bf207d6837...|   Naver|02f6364bf207d6837...|2019-09-07 18:24:...|
|  1961|0302915889fa38fe5...| Youtube|0302915889fa38fe5...| 2019-11-29 15:16:49|
|  2139|039cc47f960921e3c...|   Naver|039cc47f960921e3c...| 2019-08-01 18:19:34|
|   599|03ce0331cbd983f16...

In [4]:

spark.sql("DROP TABLE IF EXISTS bk_usc")
spark.sql("DROP TABLE IF EXISTS bk_st")

df_user_session_channel.write.mode("overwrite").bucketBy(3, "sessionid").saveAsTable("bk_usc")
df_session_timestamp.write.mode("overwrite").bucketBy(3, "sessionid").saveAsTable("bk_st")

23/10/04 02:20:03 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/04 02:20:03 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/10/04 02:20:04 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/10/04 02:20:04 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.21.0.3
23/10/04 02:20:05 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


23/10/04 02:20:07 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/10/04 02:20:07 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/10/04 02:20:07 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/04 02:20:07 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [ ]:
df_bk_usc = spark.read.table("bk_usc")
df_bk_st = spark.read.table("bk_st")

join_expr2 = df_bk_usc.sessionid == df_bk_st.sessionid
df_join2 = df_bk_usc.join(df_bk_st, join_expr2, "inner")

df_join2.show(10)

input("Waiting ...")


+------+--------------------+---------+--------------------+--------------------+
|userid|           sessionid|  channel|           sessionid|                  ts|
+------+--------------------+---------+--------------------+--------------------+
|  1491|00029153d12ae1c9a...|  Organic|00029153d12ae1c9a...|2019-10-18 14:14:...|
|  1667|000958fdaefe0dd06...|Instagram|000958fdaefe0dd06...|2019-11-02 14:52:...|
|   468|000a3f777828d2cdb...| Facebook|000a3f777828d2cdb...| 2019-11-29 16:33:55|
|   780|000c076c390a4c357...|    Naver|000c076c390a4c357...|2019-05-02 15:24:...|
|   711|00106ac9184e7d54b...|  Organic|00106ac9184e7d54b...|2019-07-25 18:14:...|
|  1653|0012a83c1e6cda9d7...|   Google|0012a83c1e6cda9d7...| 2019-08-07 16:08:41|
|    64|001a3678cacf4ea11...|Instagram|001a3678cacf4ea11...|2019-10-04 19:57:...|
|  2027|001c4bab90e1fbf3a...| Facebook|001c4bab90e1fbf3a...| 2019-11-20 20:23:10|
|   197|001d3439223b7bb23...|    Naver|001d3439223b7bb23...|2019-08-14 15:14:...|
|  1513|001f35b8